In [1]:
import pandas as pd
import numpy as np
new_lab = pd.read_csv("df_labdata.csv")
new_lab.rename(columns={"test_result": "result", "test_unit": 'Units'}, inplace = 'True')
old_lab = pd.read_csv('TMA_lab.csv')
old_lab['Lab_Date'] = pd.to_datetime(old_lab.Lab_Date)
old_lab.rename(columns={"Lab_Date": "date", "Results": "result", "Code": "test_name"}, inplace = 'True')
master = pd.read_csv("master.csv")
masnew = pd.read_csv('master_new.csv')
masnew.rename(columns={"ACCT": "TMA_Acct"}, inplace = 'True')
def num(df):
    print(len(df))
    print(len(df['TMA_Acct'].unique()))
def show(df, n):
    print(pd.DataFrame(df[df['TMA_Acct'] == n]))

In [7]:
hemo_name = ['hemoglobin', 'hemoglobin and hematocrit', 'hemoglobin,plasma'] 
new_hemo = new_lab[new_lab['test_name'].isin(hemo_name)][['TMA_Acct','date', 'test_name', 'result', 'Units']]
old_hemo = old_lab[old_lab['test_name'].isin(hemo_name)][['TMA_Acct', 'date', 'test_name', 'result', 'Units']]

In [139]:
hemo = pd.concat([old_hemo, new_hemo], axis = 0)
hemo = hemo.sort_values(['TMA_Acct', 'date'])
hemo['result'] = hemo['result'].replace(['12.0.'],'12.0')
hemo['result'] = hemo['result'].replace(['10.031.8'],'10.031')

In [140]:
delete = ['Canceled', 'Not Done', 'Comment', 'Ignore Results,', 'NR', 'Pending', 'See Note']
hemo = hemo[~hemo['result'].isin(delete)]

In [141]:
hemo['result'] = hemo['result'].astype('float')
hemo = hemo.dropna(subset = ['result'])

In [142]:
set(list(hemo['Units']))

{'G/DL', 'GM/DL', 'MG/DL', 'g/dL', 'g/dl', 'gm/dL', nan}

In [143]:
max(hemo['result'])

1414.3

In [87]:
#hemo = hemo[hemo['result'] < 414.4]

In [54]:
stage_base = stage_base.read_csv('stage_base.csv')

In [56]:
stage_base

,TMA_Acct,Age_in_2010,sex_v,race_v,hyper_v,diabe_v,stage
0,97,76,1,0,1,1,3a
1,499,82,0,0,1,0,3a
2,757,77,0,0,1,1,3a
3,1025,80,1,0,1,0,3a
4,1052,49,0,0,1,0,3a
...,...,...,...,...,...,...,...
568,50295,76,0,0,0,0,4
569,50431,88,1,0,0,0,4
570,50466,74,1,0,1,0,4
571,50618,71,1,0,0,0,4


In [144]:
print(len(hemo))
hemo = hemo[hemo['result'] < 25]
print(len(hemo))

83684
83601


In [145]:
max(hemo['result'])

24.5

In [146]:
hemo = hemo[['TMA_Acct', 'date', 'result']]
hemo['date'] = pd.to_datetime(hemo['date'])
hemo = hemo[(hemo['date'] > '2009-12-31') & (hemo['date'] < '2019-01-01')]
hemo = hemo.drop_duplicates()
hemo['date'] = pd.to_datetime(hemo['date'])
early = pd.DataFrame(hemo.groupby(['TMA_Acct']).agg({'date': [np.min]})).reset_index()
early.columns=['TMA_Acct', 'min']
hemo = pd.merge(hemo, early, how = 'left', left_on = ['TMA_Acct'], right_on = ['TMA_Acct'])
hemo['date'] = pd.to_datetime(hemo['date'])
hemo['min'] = pd.to_datetime(hemo['min'])
hemo['period'] = ((hemo['date'] - hemo['min']).dt.days / 90).astype('int')
hemo_traj = hemo[['TMA_Acct', 'result', 'period']]
hemo_traj = pd.DataFrame(hemo_traj.groupby(['TMA_Acct', 'period']).agg({'result': np.mean})).reset_index()
hemo_traj = hemo_traj.pivot(index='TMA_Acct', columns='period', values='result')
print(len(hemo_traj))
final_hemo = hemo_traj.dropna(thresh=3)
print(len(final_hemo))
for i in range(37):
    final_hemo['t' + str(i)] = i
    final_hemo.rename(columns={i: "v" + str(i)}, inplace = 'True')

10428
5081


C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\DELL\AppData\Roaming\Python\Python37\site-packages\pandas\core\frame.py:4133: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [120]:
final_hemo

period,v0,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,t27,t28,t29,t30,t31,t32,t33,t34,t35,t36
TMA_Acct,,,,,,,,,,,,,,,,,,,,,
37,8.7,9.50,NaN,9.150000,8.100000,NaN,NaN,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
64,12.7,12.30,12.5,12.500000,12.666667,NaN,NaN,12.0,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
86,10.8,10.60,10.6,10.100000,9.800000,10.20,NaN,10.1,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
90,13.5,3.82,12.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
97,16.3,NaN,NaN,NaN,14.100000,15.10,NaN,NaN,13.8,NaN,...,27,28,29,30,31,32,33,34,35,36
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52768,15.0,15.20,14.9,13.566667,NaN,NaN,NaN,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
52782,13.0,NaN,NaN,NaN,11.200000,NaN,NaN,10.4,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36
52891,12.5,12.50,12.6,NaN,NaN,12.85,12.15,NaN,NaN,NaN,...,27,28,29,30,31,32,33,34,35,36


In [147]:
stage_hemo = pd.merge(final_hemo, stage_base, left_on = 'TMA_Acct', right_on = 'TMA_Acct')
stage_hemo_3a = stage_hemo[stage_hemo['stage'] == '3a']
stage_hemo_3b = stage_hemo[stage_hemo['stage'] == '3b']
stage_hemo_4 = stage_hemo[stage_hemo['stage'] == '4']
stage_hemo_3a.to_csv('stage_hemo_3a.csv')
stage_hemo_3b.to_csv('stage_hemo_3b.csv')
stage_hemo_4.to_csv('stage_hemo_4.csv')

In [148]:
stage_hemo_3a.to_csv('stage_hemo_3a.csv')
stage_hemo_3b.to_csv('stage_hemo_3b.csv')
stage_hemo_4.to_csv('stage_hemo_4.csv')

In [149]:
len(stage_hemo_3a)

759

In [150]:
len(stage_hemo_3b)

901

In [151]:
len(stage_hemo_4)

524